In [ ]:
# Install these packages if running from colab
!pip install tensorflow-datasets --quiet
!pip install pydot --quiet
!pip install transformers --quiet

# install huggingface datasets
!pip install datasets --quiet

! pip install rouge-score nltk --quiet
! pip install huggingface_hub --quiet

     |████████████████████████████████| 5.5 MB 14.9 MB/s 
     |████████████████████████████████| 7.6 MB 50.9 MB/s 
     |████████████████████████████████| 163 kB 73.7 MB/s 
     |████████████████████████████████| 441 kB 13.7 MB/s 
     |████████████████████████████████| 95 kB 5.9 MB/s 
     |████████████████████████████████| 212 kB 66.7 MB/s 
     |████████████████████████████████| 115 kB 72.7 MB/s 
     |████████████████████████████████| 127 kB 73.1 MB/s 
     |████████████████████████████████| 115 kB 62.1 MB/s 


In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re

#let's make longer output readable without scrolling
from pprint import pprint

# the toxic parallel dataset, with rouge metric
from datasets import load_dataset, load_from_disk, load_metric, DatasetDict

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# define paths
dataset_path = 'drive/MyDrive/Colab Notebooks/w266_project_data'
csv_path = 'drive/MyDrive/Colab Notebooks/w266_project_predictions/'

#### Change these variables as needed for different model and different file name

In [ ]:
# change these variables for different models
output_file_name = 'bart_large_zsl.csv'
model_checkpoint = "facebook/bart-large"

### Import and examine our dataset

In [ ]:
# load the dataset
dataset = load_from_disk(dataset_path)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 17789
    })
    test: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 989
    })
    valid: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 988
    })
})

In [ ]:
metric = load_metric("rouge")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [ ]:
pd.DataFrame(dataset['train'])

,en_toxic_comment,en_neutral_comment
0,u 'd be surprised all the shit u think about w...,You would be supriesd all the things you think...
1,plenty of holocaust deniers actually believe t...,Plenty of Holocaust deniers actually believe t...
2,holy fuck i 'm so going,i'm so going
3,i 've driven by lots of cattle farms but holy ...,i 've driven by lots of cattle farms but this
4,so much better this shit to soft for me,So much better this is to soft for me.
...,...,...
17784,"but , thats exactly what a couple of you idiot...","But, that’s exactly what a couple of you have ..."
17785,he probably assessed the situation and decided...,He probably assessed the situation and decided...
17786,brah im fucked up over here .,Brah I am feeling bad over here
17787,shit with my friends and not being able to tru...,with my friends and not being able to trust th...


## load BART large pretrain model

In [ ]:
# model_checkpoint = "facebook/bart-large"

In [ ]:
from transformers import BartTokenizer, TFBartForConditionalGeneration

model = TFBartForConditionalGeneration.from_pretrained(model_checkpoint)
tokenizer = BartTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

In [ ]:
model.summary()

Model: "tf_bart_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFBartMainLayer)     multiple                  406291456 
                                                                 
 final_logits_bias (BiasLaye  multiple                 50265     
 r)                                                              
                                                                 
Total params: 406,341,721
Trainable params: 406,291,456
Non-trainable params: 50,265
_________________________________________________________________


### Preprocess: Encode detox dataset with BART tokenizer

In [ ]:
## Encode detox train_texts with BART tokenizer
max_length = 25
#max_label_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples['en_toxic_comment']]
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['en_neutral_comment'], max_length=max_length, truncation=True, padding=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# test the preprocess function
preprocess_function(dataset['train'][:2])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


{'input_ids': [[0, 257, 128, 417, 28, 3911, 70, 5, 15328, 1717, 206, 59, 77, 1717, 1236, 687, 579, 2582, 179, 89, 2], [0, 2911, 24997, 9, 18701, 43328, 3069, 4733, 888, 679, 5, 5373, 15328, 51, 224, 479, 2, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]], 'labels': [[0, 1185, 74, 28, 28836, 4458, 417, 70, 5, 383, 47, 206, 59, 77, 47, 32, 95, 2828, 89, 4, 2], [0, 16213, 24997, 9, 13903, 3069, 4733, 888, 679, 5, 383, 51, 224, 4, 2, 1, 1, 1, 1, 1, 1]]}

In [ ]:
# tokenize the detox dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

### no fine-tuning, zero-shot. Try on one example

In [ ]:
input_tokenized = tokenizer([dataset['test']['en_toxic_comment'][1]], return_tensors="tf").input_ids
summary_ids = model.generate(input_tokenized, num_beams=2, min_length=0, max_length=25)

/usr/local/lib/python3.7/dist-packages/transformers/generation_tf_utils.py:1699: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [ ]:
prediction = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
reference = [dataset['test']['en_neutral_comment'][1]]

In [ ]:
print([dataset['test']['en_toxic_comment'][1]])
print(prediction)
print(reference)

['all she has to do is smile and keep her cool with slobbering old wrinkled hubby .']
['all she has to do is smile and keep her cool with slobbering old']
['All she has to do is smile and keep her cool with old aged hubby']


In [ ]:
rouge_results = metric.compute(predictions=prediction,
                               references=reference)
pprint(rouge_results, compact=True)  

{'rouge1': AggregateScore(low=Score(precision=0.9285714285714286, recall=0.8666666666666667, fmeasure=0.896551724137931), mid=Score(precision=0.9285714285714286, recall=0.8666666666666667, fmeasure=0.896551724137931), high=Score(precision=0.9285714285714286, recall=0.8666666666666667, fmeasure=0.896551724137931)),
 'rouge2': AggregateScore(low=Score(precision=0.8461538461538461, recall=0.7857142857142857, fmeasure=0.8148148148148148), mid=Score(precision=0.8461538461538461, recall=0.7857142857142857, fmeasure=0.8148148148148148), high=Score(precision=0.8461538461538461, recall=0.7857142857142857, fmeasure=0.8148148148148148)),
 'rougeL': AggregateScore(low=Score(precision=0.9285714285714286, recall=0.8666666666666667, fmeasure=0.896551724137931), mid=Score(precision=0.9285714285714286, recall=0.8666666666666667, fmeasure=0.896551724137931), high=Score(precision=0.9285714285714286, recall=0.8666666666666667, fmeasure=0.896551724137931)),
 'rougeLsum': AggregateScore(low=Score(precision=

### zero-shot learning (ZSL) on the test set

In [ ]:
test_predictions = []
test_references = []
test_len = len(dataset['test']['en_toxic_comment'])

for i in range(test_len):
  input_tokenized = tokenizer([dataset['test']['en_toxic_comment'][i]], return_tensors="tf").input_ids
  summary_ids = model.generate(input_tokenized, num_beams=2, min_length=0, max_length=25)
  
  prediction = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  reference = [dataset['test']['en_neutral_comment'][i]]
  
  test_predictions.insert(i, (prediction))
  test_references.insert(i, (reference))

  if i % 50 == 0:
    print('complete', i, '/', test_len)
#print(len(val_references))

complete 0 / 989
complete 50 / 989
complete 100 / 989
complete 150 / 989
complete 200 / 989
complete 250 / 989
complete 300 / 989
complete 350 / 989
complete 400 / 989
complete 450 / 989
complete 500 / 989
complete 550 / 989
complete 600 / 989
complete 650 / 989
complete 700 / 989
complete 750 / 989
complete 800 / 989
complete 850 / 989
complete 900 / 989
complete 950 / 989


In [ ]:
# dictionary of lists  
dict = {'test_predictions': test_predictions, 'test_references': test_references}  
       
df = pd.DataFrame(dict) 

In [ ]:
# saving the output dataframe to a csv file
# output_file_name = 'bart_large_zsl.csv'
df.to_csv(csv_path + output_file_name, index = False) 

In [ ]:
# read data from csv file
df_bart_predictions = pd.read_csv(csv_path + output_file_name)

# display data frame
print(df_bart_predictions.head())

# display shape
print(df_bart_predictions.shape)

# display columns
print(df_bart_predictions.columns)

                                    test_predictions  \
0  ['holy shit , the 61 61 view must be hilarious...   
1  ['all she has to do is smile and keep her cool...   
2  ['can we apply that litmus test to muslims tha...   
3                  ['thats fucking it , im joining']   
4  ['the residents of davis have a collective sti...   

                                     test_references  
0                    ['The 6161 view must be funny']  
1  ['All she has to do is smile and keep her cool...  
2  ['can we apply that litmus test to muslims tha...  
3                        ["That's it! I'm joining."]  
4    ['The residents of Davis are not so pleasant.']  
(989, 2)
Index(['test_predictions', 'test_references'], dtype='object')


In [ ]:
rouge_results = metric.compute(predictions=df_bart_predictions['test_predictions'],
                               references=df_bart_predictions['test_references'])
pprint(rouge_results, compact=True)  

{'rouge1': AggregateScore(low=Score(precision=0.6896642862798331, recall=0.7828601295905133, fmeasure=0.7262427176079935), mid=Score(precision=0.7020906597829966, recall=0.7958200164850839, fmeasure=0.7385980717605785), high=Score(precision=0.7151288794668367, recall=0.8096807859465145, fmeasure=0.7517866290158348)),
 'rouge2': AggregateScore(low=Score(precision=0.5380319861298132, recall=0.6147364694653451, fmeasure=0.567003696423101), mid=Score(precision=0.5529837417666037, recall=0.631022322848217, fmeasure=0.583034266720047), high=Score(precision=0.5690282793967072, recall=0.6479271417409704, fmeasure=0.5992542590043356)),
 'rougeL': AggregateScore(low=Score(precision=0.6847182148288807, recall=0.7768002853221235, fmeasure=0.7207770894512548), mid=Score(precision=0.698404471692255, recall=0.791076360862683, fmeasure=0.7345244045058252), high=Score(precision=0.7115563212408467, recall=0.80424477694284, fmeasure=0.7469682040509757)),
 'rougeLsum': AggregateScore(low=Score(precision=0

In [ ]:
pd.DataFrame.from_dict(rouge_results)

,rouge1,rouge2,rougeL,rougeLsum
0,"(0.6896642862798331, 0.7828601295905133, 0.726...","(0.5380319861298132, 0.6147364694653451, 0.567...","(0.6847182148288807, 0.7768002853221235, 0.720...","(0.6848029240312172, 0.777461291828038, 0.7214..."
1,"(0.7020906597829966, 0.7958200164850839, 0.738...","(0.5529837417666037, 0.631022322848217, 0.5830...","(0.698404471692255, 0.791076360862683, 0.73452...","(0.6984377919443048, 0.7908417675411488, 0.734..."
2,"(0.7151288794668367, 0.8096807859465145, 0.751...","(0.5690282793967072, 0.6479271417409704, 0.599...","(0.7115563212408467, 0.80424477694284, 0.74696...","(0.7112521077943483, 0.8053465213951827, 0.746..."


In [ ]:
pd.concat({k: pd.DataFrame(v) for k, v in rouge_results.items()})

rouge1     0    (0.6896642862798331, 0.7828601295905133, 0.726...
           1    (0.7020906597829966, 0.7958200164850839, 0.738...
           2    (0.7151288794668367, 0.8096807859465145, 0.751...
rouge2     0    (0.5380319861298132, 0.6147364694653451, 0.567...
           1    (0.5529837417666037, 0.631022322848217, 0.5830...
           2    (0.5690282793967072, 0.6479271417409704, 0.599...
rougeL     0    (0.6847182148288807, 0.7768002853221235, 0.720...
           1    (0.698404471692255, 0.791076360862683, 0.73452...
           2    (0.7115563212408467, 0.80424477694284, 0.74696...
rougeLsum  0    (0.6848029240312172, 0.777461291828038, 0.7214...
           1    (0.6984377919443048, 0.7908417675411488, 0.734...
           2    (0.7112521077943483, 0.8053465213951827, 0.746...
dtype: object